# Import thư viện

In [ ]:
import numpy as np 
import pandas as pd
import queue
import re
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
import google.generativeai as genai

# Các hàm liên quan đến Gemini API

In [2]:
API_list = [
    "API-KEY1",
    "API-KEY2",
    "API-KEY3",
    "API-KEY4",
    "API-KEY5",
]

model_list = [
    "gemini-1.5-flash",
    "gemini-1.5-flash-8b",
    "gemini-1.0-pro"
]

API_queue = queue.Queue()
for model in model_list:
    for api in API_list:
        API_queue.put((model, api))

In [3]:
# Get model with API
def get_item():
    model, api = API_queue.get()
    API_queue.put((model, api))
    return model, api

# Config API function
def config_API(API_KEY):
    genai.configure(api_key=API_KEY)
    
# Request function
def request(API_KEY, model_name="gemini-1.0-pro", prompt=None):
    try:
        if prompt is not None:
            config_API(API_KEY)
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            return response.text
        else:
            print("Prompt must be NOT None!")
            return None
    except:
        print("\nSomething went wrong! Try again!")
        return None
    
# clear response function
def clear_response(response):
    # Thay thế \n\n bằng \n
    response = re.sub(r'\n\n+', '\n', response)
    
    # Loại bỏ markdown như ** và *
    response = re.sub(r'\*\*|\*', '', response)
    
    return response

# Thiết kế Prompt - Prompting

In [4]:
template = """Hướng dẫn: Bạn có vai trò tăng cường dữ liệu hội thoại giữa bác sĩ và người dùng. Hãy tăng cường dữ liệu bằng cách tạo ra nhiều kịch bản liên quan với câu hỏi và cách trả lời đa dạng dựa trên hội thoại mẫu mà người dùng cung cấp.
Lưu ý: Mỗi đoạn hội thoại đều được bắt đầu bằng [START] và kết thúc là [END] không cần markdown, trình bày càng dài càng đủ ý càng tốt.
Hội thoại được cung cấp: {}
Hãy tăng cường dữ liệu lên 5 lần
{}"""

example = """Ví dụ bạn sẽ sinh ra giống như [START][{"role": "Người dùng", "message":...},{"role": "Bác sĩ", ...}][END] thêm 5 lần như vậy (Không được thêm ```json hay bất cứ thứ gì)"""

# Load dữ liệu Web Crawling

In [ ]:
df = pd.read_csv("data_crawling.csv")
df.head()

# Một số hàm cần thiết

In [6]:
# extract text in response [START]...[END]
def extract_response(response):
    # Sử dụng regex để tìm tất cả các chuỗi nằm giữa [START] và [END]
    pattern = r'\[START\](.*?)\[END\]'
    conversations = re.findall(pattern, response, re.DOTALL)  # re.DOTALL để match cả xuống dòng

    # Loại bỏ các khoảng trắng thừa đầu và cuối chuỗi
    conversations = [conv.strip() for conv in conversations]
    
    return conversations
    
# Save csv function
def save_csv(data:dict, path:str):
    df = pd.DataFrame(data)
    df.to_csv(path, index=False)

# Data Augmentation

In [7]:
data_dict = {
    "conversation": []
}

def aug(conversation):
    prompt = template.format(conversation,example)
    while True:
        model, api = get_item()
        response = request(api, model, prompt)
        if response is not None:
            break
        time.sleep(2)
        
    response = clear_response(response)
    aug_conversations = extract_response(response)
    for aug_conver in aug_conversations:
        if aug_conver[0] != "[":
            aug_conver = "[" + aug_conver
        if aug_conver[-1] != "]":
            aug_conver = aug_conver + "]"
        data_dict["conversation"].append(aug_conver)

def data_augmentation():
    batch = []
    for conversation in tqdm(df["conversation"].iloc[431:], desc="Data Augmentation"):
        batch.append(conversation)
        if len(batch) >= 10:
            for i in range(10):
                with ThreadPoolExecutor(max_workers=5) as executor:
                    results = list(executor.map(aug, batch))
                save_csv(data_dict, "data_augmentation2.csv")
            batch = []
    print("DONE!")
            

In [ ]:
data_augmentation()